In [1]:
import h5py
import numpy as np
import pandas as pd
from sklearn.utils import resample
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../Shared')
import processing, Model, Visualization
from sklearn.model_selection import train_test_split
import modules

In [4]:
def Run_one_subject(F_name, total_sessions):
    base_path = 'D:/Data/Gait-EMG/Mypredict/'
    f = h5py.File(base_path+f"{F_name}", "r")

    key_lst = list(f['Day_1'].keys())

    Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']
    Sel_feat_idx = 4

    window_size, step_size = 200, 10

    X_lst, y_lst = [], []
    for trials in key_lst[1:total_sessions+1]:
        print("\n\n===================================", trials, "===================================\n")
        trial = f["Day_1"][trials]

        # 1.
        label, valid_idx = modules.label_extraction(trial)
        emg_keys = [k for k in trial.keys() if k.startswith("EMG_")]
        X_emg = modules.stack_signals(trial, emg_keys, valid_idx)

        # 2.
        balanced_idx = modules.Downsample_to_balance_class(label)
        X_emg_bal, y_bal = X_emg[balanced_idx], label[balanced_idx]

        #X_emg_bal, y_bal = random_downsample(X_emg_bal, y_bal, fraction=0.5)
        #print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

        num_channels = X_emg.shape[1]
        X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal, window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

        all_X, all_y = modules.get_X_y_WL_only(X_tmp, y_tmp)
        all_y = modules.y_change_to_int(all_y)
        all_X = all_X.reshape(-1, num_channels, 1)

        print("\t EMG:", all_X.shape, "y:", all_y.shape)
        X_lst.append(all_X)
        y_lst.append(all_y)

    final_X = np.concatenate(X_lst, axis=0)
    final_y = np.concatenate(y_lst, axis=0)
    print("final X:", final_X.shape, "y:", final_y.shape)

    return final_X, final_y


In [10]:
X, y = [], []
fnames = [f'MP10{i}.hdf5' for i in range(1,10)] + ["MP110.hdf5"]

for i in range(len(fnames)):
    X_tmp, y_tmp = Run_one_subject(F_name=f"{fnames[i]}", total_sessions=5)
    X.append(X_tmp)
    y.append(y_tmp)



=================================== Trial_01 ===================================

Unique labels: [0.  1.  2.  3.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (4031, 16, 1) y: (4031,)


=================================== Trial_02 ===================================

Unique labels: [0.  1.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
	 EMG: (3021, 16, 1) y: (3021,)


=================================== Trial_03 ===================================

Unique labels: [0.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (2981, 16, 1) y: (2981,)


=================================== Trial_04 ===================================

Unique labels: [0.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (2831, 16,

In [16]:
#Training time:
ACC_All = []
for i in range(len(fnames)):
    print(f"Target : {fnames[i]}")
    X_target, y_target = X[i], y[i]
    X_lst, y_lst, Sub_acc = [], [], []

    for j in range(len(fnames)):
        if i == j:
            continue
        else:
            X_lst.append(X[i])
            y_lst.append(y[i])
            X_train, y_train = np.concatenate(X_lst, axis=0), np.concatenate(y_lst, axis=0)

            model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
            history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=0)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)
            print(f"\t Subject added : {fnames[j]} - X_train: {X_train.shape}, y_train: {y_train.shape}  ===> accuracy : {acc*100:.2f}")
            Sub_acc.append(acc)
    print("\n\n")
    ACC_All.append(Sub_acc)

Target : MP101.hdf5
	 Subject added : MP102.hdf5 - X_train: (15245, 16, 1), y_train: (15245,)  ===> accuracy : 73.30
	 Subject added : MP103.hdf5 - X_train: (30490, 16, 1), y_train: (30490,)  ===> accuracy : 86.17
	 Subject added : MP104.hdf5 - X_train: (45735, 16, 1), y_train: (45735,)  ===> accuracy : 86.30
	 Subject added : MP105.hdf5 - X_train: (60980, 16, 1), y_train: (60980,)  ===> accuracy : 89.92
	 Subject added : MP106.hdf5 - X_train: (76225, 16, 1), y_train: (76225,)  ===> accuracy : 94.69
	 Subject added : MP107.hdf5 - X_train: (91470, 16, 1), y_train: (91470,)  ===> accuracy : 92.00
	 Subject added : MP108.hdf5 - X_train: (106715, 16, 1), y_train: (106715,)  ===> accuracy : 95.10
	 Subject added : MP109.hdf5 - X_train: (121960, 16, 1), y_train: (121960,)  ===> accuracy : 96.40
	 Subject added : MP110.hdf5 - X_train: (137205, 16, 1), y_train: (137205,)  ===> accuracy : 95.43



Target : MP102.hdf5
	 Subject added : MP101.hdf5 - X_train: (16055, 16, 1), y_train: (16055,)  ===

In [17]:
ACC_All

[[0.7329616546630859,
  0.861659586429596,
  0.8630370497703552,
  0.8992456793785095,
  0.9469334483146667,
  0.9199737906455994,
  0.9510003328323364,
  0.963988184928894,
  0.9542800784111023],
 [0.8936780095100403,
  0.9665524959564209,
  0.9744628071784973,
  0.9747119545936584,
  0.9845530986785889,
  0.987605094909668,
  0.9921519756317139,
  0.9927748441696167,
  0.9913422465324402],
 [0.8399999737739563,
  0.900357723236084,
  0.9309918880462646,
  0.9529756307601929,
  0.9607154726982117,
  0.9681300520896912,
  0.9689105749130249,
  0.9717723727226257,
  0.9773008227348328],
 [0.9403754472732544,
  0.9574292302131653,
  0.967674195766449,
  0.9771555662155151,
  0.9811645150184631,
  0.9796372652053833,
  0.9816099405288696,
  0.9853643178939819,
  0.9902640581130981],
 [0.8687152862548828,
  0.9379488825798035,
  0.9574231505393982,
  0.9723805785179138,
  0.9771195650100708,
  0.9663828015327454,
  0.9754165410995483,
  0.9843021035194397,
  0.9877082705497742],
 [0.914731

In [22]:
#pd.DataFrame(np.array(ACC_All).T, columns=fnames).to_csv("./leave_one_out_inter_subject_cumulative_fashion.csv", index=False)